# 17 Category Flower Dataset

## Part 1 - Observing Results from 2006 Paper
Observe the results from the 2006 paper using the precomputed feature distance matrices.

### Imports

In [2]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
gdrive_dir = '/gdrive'
drive.mount(gdrive_dir)
gdrive_dir = gdrive_dir+'/My Drive/Colab Notebooks/'

### Load files

In [3]:
datasplits = scipy.io.loadmat('datasplits.mat')
datasplits.keys() # Observe keys

dict_keys(['__header__', '__version__', '__globals__', 'trn1', 'trn2', 'trn3', 'tst1', 'tst2', 'tst3', 'val3', 'val2', 'val1'])

In [4]:
datasplits['trn1'].shape # What do the train splits look like?

(1, 680)

In [5]:
datasplits['tst1'].shape # What do the test splits look like?

(1, 340)

In [6]:
set(datasplits['trn1'][0]) == set(datasplits['trn2'][0]) # Are the splits permuations of the same set?

False

In [7]:
distmtxs = scipy.io.loadmat('distancematrices17gcfeat06.mat')
distmtxs.keys() # Observe keys

dict_keys(['__header__', '__version__', '__globals__', 'D_colourgc', 'D_texturegc', 'D_shapegc'])

In [8]:
distmtxs['D_shapegc'].shape # What do the distance matrices look like

(1360, 1360)

In [9]:
labels = np.repeat(range(17), 80) # Create a label array based on the ordering of the images
labels.shape

(1360,)

### Train kNN classifier

In [10]:
train = datasplits['trn1'][0] # Only using one of 3 possible splits for this preliminary test
test = datasplits['tst1'][0]

In [11]:
# Verify the paper's results using the precomputed distance matrices
model = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
model.fit(distmtxs['D_shapegc'][train, :][:, train], labels[train])
y_pred = model.predict(distmtxs['D_shapegc'][test, :][:, train])

In [12]:
accuracy_score(labels[test], y_pred)

0.5294117647058824

Not sure why the accuracy is so low. Could have something to do with multiple hypotheses as discussed in the paper which I'm not sure I understand. Not going to worry for now since this was just meant as a sanity check.

## Part 2 - Transfer Learning with PyTorch
Use transfer learning, freezing convolutional layers but training new classification layers to work with flower images.

### Imports

In [13]:
%matplotlib inline
import torch
import torchvision
import cv2
from PIL import Image

from torch.utils.data import Subset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
from collections import OrderedDict
from torch import nn

In [14]:
# Transformation to the input ResNet expects
resnet_mean = np.array([0.485, 0.456, 0.406])
resnet_std = np.array([0.229, 0.224, 0.225])
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=resnet_mean, std=resnet_std),
])

### Load images
**NB** I restructed nature of the jpg directory that was downloaded from http://www.robots.ox.ac.uk/~vgg/data/flowers/17/index.html to make it easier to use Torch's ImageLoader

In [15]:
 # All the images
fullset = ImageFolder('jpg', transform=preprocess)

# Split keys - names of splits (might as well use splits already computed in paper)
splitkeys = ['trn1', 'trn2', 'trn3', 'val1', 'val2', 'val3', 'tst1', 'tst2', 'tst3', ]

# Image datasets as given by the paper's splits
datasets = {key : Subset(fullset, datasplits[key][0]) for key in splitkeys} 

# Image loaders from the datasets
dataloaders = {key : DataLoader(datasets[key], batch_size=8, shuffle=False, num_workers=4) for key in splitkeys}
print(f"Train set 1: {len(datasets['trn1'])} images / {len(dataloaders['trn1'])} batches")
print(f"Valid set 1: {len(datasets['val1'])} images / {len(dataloaders['val1'])} batches")
print(f"Test set 1:  {len(datasets['tst1'])} images / {len(dataloaders['tst1'])} batches")

Train set 1: 680 images / 85 batches
Valid set 1: 340 images / 43 batches
Test set 1:  340 images / 43 batches


In [16]:
class_names = fullset.classes
class_names

['bluebell',
 'buttercup',
 'coltsfoot',
 'cowslip',
 'crocus',
 'daffodil',
 'daisy',
 'dandelion',
 'fritillary',
 'iris',
 'lilyvalley',
 'pansy',
 'snowdrop',
 'sunflower',
 'tigerlily',
 'tulip',
 'windflower']

### View some images
**This is mostly copied from the PyTorch tutorial**  https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [17]:
'''
def imshow(img):
    # Un-normalize
    for i in range(img.shape[0]):
        img[i] = img[i] * resnet_std[i] + resnet_mean[i]
    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images
dataiter = iter(dataloaders['trn1'])
images, labels = dataiter.next()

# Show images
#fig = plt.figure(figsize=(15, 120))
imshow(torchvision.utils.make_grid(images))
'''

"\ndef imshow(img):\n    # Un-normalize\n    for i in range(img.shape[0]):\n        img[i] = img[i] * resnet_std[i] + resnet_mean[i]\n    \n    npimg = img.numpy()\n    plt.imshow(np.transpose(npimg, (1, 2, 0)))\n    plt.show()\n\n# Get some random training images\ndataiter = iter(dataloaders['trn1'])\nimages, labels = dataiter.next()\n\n# Show images\n#fig = plt.figure(figsize=(15, 120))\nimshow(torchvision.utils.make_grid(images))\n"

### Configure transfer learning CNN
Using becase its pretty light and easy to use with transfer learning

In [18]:
#  Match the FC input layer (25088) to the output number of classes (17) with one hidden layer of 4096
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 17)),
                          ('output', nn.LogSoftmax(dim=1))
]))

In [ ]:
model = models.vgg19(pretrained=True)

In [ ]:
# Freeze the pretrained weights
for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
# Replace the classifier portion of VGG19
model.classifier = classifier

### Train the model

In [ ]:
def train_model(model, criteria, optimizer, scheduler, datasplit=1,   
                                      num_epochs=20, device='cuda'):
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {} / {}'.format(epoch, num_epochs - 1))
        
        # Iterate through train and validation phases
        for phase in ['trn', 'val']:
            if phase is 'trn':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase + datasplit]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Set parameter gradients to zero
                optimizer.zero_grad()

                # Forward propagatation
                with torch.set_grad_enabled(phase is 'trn'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward propagation
                    # Optimize only if in training phase
                    if phase is 'trn':
                        loss.backward()
                        optimizer.step()

                # Compute statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f}, Accuracy: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Save this model 
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                wts_best = copy.deepcopy(model.state_dict())

    # Load the best model
    model.load_state_dict(best_model)
    return model

In [ ]:
# Following hyperparameters are used

# Negative log likelihood loss - good with softmax output
criteria = nn.NLLLoss()

# Use Adam optimizer
optim = optim.Adam(model.classifier.parameters(), lr=0.001)

# Decrease LR by a factor of 4 for each optimization
sched = lr_scheduler.StepLR(optim, step_size=4, gamma=0.1)

In [ ]:
model_trained = train_model(model, criteria, optim, sched, eps)